In [ ]:
import config
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from IPython.display import display
import json
import datetime
import math

from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_fscore_support, f1_score, cohen_kappa_score, average_precision_score

In [ ]:
STAGE = 'ss'
MOD_PREFIX = "mod_smallimg3"
NEPOCH = 'latest'


DATAPATH = config.OUTPUT_PATH
base_dir = config.RAW_DATA_PATH
img_path = config.SCHULTHESS_DATAPATH
proc_dir = config.PROC_DATA_PATH

folder = "2025-11-19_hdbscan"
run = "run10"  #"run175"

anomalyscore_metric = "centre_mean"
cluster_col = "cluster_label"

In [ ]:
today = datetime.date.today()
folder_date = folder.split('_')[0]

filepath = os.path.join(proc_dir, folder, "pipeline", run)
save_path = os.path.join(filepath, "img")
os.makedirs(save_path, exist_ok=True)

# try:
#     hdbscan_df = pd.read_csv(os.path.join(filepath, f'pipeline_{run}_umap_hdbscan_scaled_allpoints_wKL.csv'))
#     kl = pd.read_csv(os.path.join(base_dir,  "brul_knee_annotations.csv"))
#     kl2 = pd.read_csv(os.path.join(base_dir, "rosand1_knee_annotations.csv"))
# except:
hdbscan_df = pd.read_csv(os.path.join(filepath, f'pipeline_{run}_umap_hdbscan_scaled.csv'))
#.merge(kl_df, left_on = 'id', right_on='name', how='left', validate='one_to_one')
kl = pd.read_csv(os.path.join(base_dir,  "brul_knee_annotations.csv"))
kl2 = pd.read_csv(os.path.join(base_dir, "rosand1_knee_annotations.csv"))
hdbscan_df = hdbscan_df.merge(kl, left_on = 'id', right_on='name', how='left', validate='one_to_one')

with open(os.path.join(filepath, f'pipeline_{run}_umap_hdbscan_scaled_model_info.json')) as f:
    model_info= json.load(f)

In [ ]:
hdbscan_df = hdbscan_df.merge(kl2, left_on = 'id', right_on='name', how='left', validate='one_to_one')

In [ ]:
# rename KL-Score_y to KL-Score2
hdbscan_df = hdbscan_df.rename(columns={"KL-Score_x": "KL-Score", "KL-Score_y": "KL-Score2"})

try:
    hdbscan_df.drop(columns=['Unnamed: 0'], inplace=True)
except:
    pass

In [ ]:
hdbscan_df.columns
## need to add mri data

In [ ]:
df = pd.read_csv(os.path.join(DATAPATH, 'outputs', 'dfs', 'ss', 'mod_smallimg3_ss_aggregated_scores.csv'))

In [ ]:
df['id_temp'] = df['id'].apply(lambda x: x.split('/')[-1])

In [ ]:
df['id'] = df['id_temp'].apply(lambda x: x.split('.')[0])

In [ ]:
df.drop(columns=['id_temp'], inplace=True)

In [ ]:
hdbscan_df.head()

In [ ]:
hdbscan_df[hdbscan_df['cluster_label'].isna()]

In [ ]:
df_hdbscan = df.merge(hdbscan_df[['id', 'cluster_label']], on='id', how='left', validate='one_to_one')


In [ ]:
df_hdbscan[df_hdbscan['cluster_label'].isna()]

In [ ]:
print(hdbscan_df.shape)
print(df.shape)

In [ ]:
df_hdbscan[df_hdbscan['cluster_label'].isna()]['cluster_label'] = -1


In [ ]:
df_hdbscan = df_hdbscan.merge(kl[['name', 'KL-Score']], left_on='id', right_on='name', how='left', validate='one_to_one')
df_hdbscan = df_hdbscan.merge(kl2[['name', 'KL-Score']], left_on='id', right_on='name', how='left', validate='one_to_one', suffixes=('', '2'))

In [ ]:
# if 'KL-Score'  is na, fill with 'KL-Score2'
df_hdbscan['KL-Score'] = df_hdbscan['KL-Score'].fillna(df_hdbscan['KL-Score2'])
df_hdbscan['KL-Score2'] = df_hdbscan['KL-Score2'].fillna(df_hdbscan['KL-Score'])

In [ ]:
df_hdbscan[df_hdbscan['KL-Score2'].isna()]

In [ ]:
# see if kl has 'IM3022_2_right'

kl['name'].str.contains('IM3022_2_right').any()
kl2['name'].str.contains('IM3022_2_right').any()

# Quick KL-comparison

In [ ]:
kl['KL-Score'].value_counts().sort_index()

In [ ]:
kl2['KL-Score'].value_counts().sort_index()

# Comparison KL-Score (AUC/SRC)

In [ ]:
def get_metrics(df, score, label_name = 'KL-Score'):

    res = stats.spearmanr(df[score].tolist(), df[label_name].tolist())

    df['binary_label'] = 0
    df.loc[df[label_name] > 0, 'binary_label'] = 1
    fpr, tpr, thresholds = roc_curve(np.array(df['binary_label']),np.array(df[score]))
    auc = metrics.auc(fpr, tpr)


    df['binary_label'] = 0
    df.loc[df[label_name] > 1, 'binary_label'] = 1

    fpr, tpr, thresholds = roc_curve(np.array(df['binary_label']),np.array(df[score]))
    auc_mid = metrics.auc(fpr, tpr)


    df['binary_label'] = 0
    df.loc[df[label_name] > 2, 'binary_label'] = 1
    fpr, tpr, thresholds = roc_curve(np.array(df['binary_label']),np.array(df[score]))
    auc_mid2 = metrics.auc(fpr, tpr)


    df['binary_label'] = 0
    df.loc[df[label_name] == 4, 'binary_label'] = 1
    fpr, tpr, thresholds = roc_curve(np.array(df['binary_label']),np.array(df[score]))
    auc_sev = metrics.auc(fpr, tpr)



    return res[0], auc, auc_mid, auc_mid2, auc_sev

In [ ]:
hdbscan_df['KL-Score2'] = hdbscan_df['KL-Score2'].fillna(hdbscan_df['KL-Score'])

## Compare Cluster Label to KL-Score

In [ ]:
metrics_clkl = get_metrics(hdbscan_df, 'cluster_label', label_name = 'KL-Score')
metrics_clkl2 = get_metrics(hdbscan_df, 'cluster_label', label_name = 'KL-Score2')

In [ ]:
print(metrics_clkl)

In [ ]:
print(metrics_clkl2)

'Osteophytes', 'Joint-Space-Narrowing', 'pain'

In [ ]:
as_col = df.iloc[:, 1:-3].columns

# use get_metrics to get src, auc, auc_mid, auc_mid2, auc_sev for all as_col and then calculate mean and std of those values
results = {}
for col in as_col:
    results[col] = get_metrics(df_hdbscan, col, label_name = 'KL-Score')
results_df = pd.DataFrame(results).T
results_df.columns = ['spearmanr', 'auc', 'auc_mid', 'auc_mid2', 'auc_sev']
results_df['spearmanr_mean'] = results_df['spearmanr'].mean()
results_df['spearmanr_std'] = results_df['spearmanr'].std()
results_df['auc_mean'] = results_df['auc'].mean()
results_df['auc_std'] = results_df['auc'].std()
results_df['auc_mid_mean'] = results_df['auc_mid'].mean()
results_df['auc_mid_std'] = results_df['auc_mid'].std()
results_df['auc_mid2_mean'] = results_df['auc_mid2'].mean()
results_df['auc_mid2_std'] = results_df['auc_mid2'].std()
results_df['auc_sev_mean'] = results_df['auc_sev'].mean()
results_df['auc_sev_std'] = results_df['auc_sev'].std()

In [ ]:
results_df.head()